In [63]:
import json
import csv
import pandas as pd
from pandas import DataFrame, Series
import dateutil.parser
from dateutil.tz import gettz
import datetime
import pytz
from functools import reduce

In [66]:
tone_names = ['Sadness', 'Analytical', 'Joy', 'Fear', 'Tentative', 'Anger', 'Confident']
tone_names

['Sadness', 'Analytical', 'Joy', 'Fear', 'Tentative', 'Anger', 'Confident']

In [40]:
#keylogger DF

def roundTime(dt):
    dt = dt - datetime.timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)
    return dt

with open('../keylogger/logs/log.json', 'r') as f:
    keyloggerData = json.load(f)

tzinfos = { "EDT" : gettz("America/New_York") }

def extract_keyloggerData(data):
    results = []

    for d in data:
        result = [0]*(len(tone_names) + 1)
        tones = d['document_tone']['tones']
        for i in range(len(tones)):
            score = tones[i]['score']
            tone_name = tones[i]['tone_name']
            tone_index = tone_names.index(tone_name)
            result[tone_index] = score            

        time = dateutil.parser.parse(d['time'] + " EDT", tzinfos=tzinfos)
        result[-1] = time
        results.append(tuple(result))
    return results

keyloggerDF = DataFrame(extract_keyloggerData(keyloggerData),
                        columns=[tone_name+"_score" for tone_name in tone_names] + ['time'])
#keyloggerDF.time = keyloggerDF.time.apply(roundTime)
#keyloggerDF.time.apply(roundTime)
keyloggerDF

,Sadness_score,Analytical_score,Joy_score,Fear_score,Tentative_score,Anger_score,Confident_score,time
0,0.000000,0.000000,0.638807,0.000000,0.000000,0.000000,0.667008,2018-03-07 22:52:33.301586-05:00
1,0.000000,0.000000,0.535479,0.000000,0.000000,0.000000,0.000000,2018-03-07 23:52:33.926227-05:00
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2018-03-08 00:52:34.524265-05:00
3,0.000000,0.000000,0.594168,0.000000,0.000000,0.000000,0.000000,2018-03-08 01:52:35.028535-05:00
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.780034,2018-03-08 08:52:37.222100-05:00
5,0.000000,0.000000,0.567961,0.000000,0.000000,0.000000,0.000000,2018-03-09 14:32:14.569387-05:00
6,0.000000,0.000000,0.577478,0.000000,0.000000,0.000000,0.000000,2018-03-09 15:46:45.864690-05:00
7,0.000000,0.000000,0.579676,0.000000,0.000000,0.555240,0.000000,2018-03-10 01:50:06.965740-05:00
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.821033,2018-03-10 21:03:45.471070-05:00
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.878843,2018-03-10 22:03:46.132751-05:00


In [45]:
#affectiva DF

local_tz = gettz('America/New_York')

def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=local_tz)

with open('../affectiva/analyses/merged_file.json', 'r') as f:
    affectivaData = json.load(f)

for x in range(0, len(affectivaData)):
    affectivaData[x]['time'] = utc_to_local(datetime.datetime.fromtimestamp((affectivaData[x]['time']/ 1e3)))

affectivaDF = DataFrame(affectivaData)
# affectivaDF.time = affectivaDF.time.apply(roundTime)
# affectivaDF.time.apply(roundTime)
#affectivaDF.emoji = affectivaDF.emoji.apply(lambda x: ",".join(x))
del affectivaDF['emoji']
#affectivaDF.emotions = affectivaDF.emotions.apply(lambda x: ",".join(x))
del affectivaDF['emotions']
affectivaDF

,blinks,max_attention,max_engagement,max_valence,min_attention,min_engagement,min_valence,time
0,2,98.778671,0.608366,0.000000,95.007645,0.079540,-1.742792,2018-04-26 17:39:34.922000-04:00
1,1,98.783875,99.781586,0.000000,40.514740,0.000000,-23.904726,2018-04-13 16:37:47.068000-04:00
2,18,98.778580,99.919434,85.839043,96.775177,0.080415,-18.714973,2018-04-15 13:19:13.564000-04:00
3,0,98.782219,31.617508,0.000000,95.013878,0.079229,-22.398460,2018-04-15 17:59:22.604000-04:00
4,9,98.767960,49.626522,0.000000,92.278549,0.080189,-19.755705,2018-04-15 22:22:18.111000-04:00
5,0,98.786530,9.790726,0.000000,84.819733,0.079817,-0.357021,2018-04-16 17:21:57.640000-04:00
6,10,98.776245,99.866013,0.000000,83.653297,0.079696,-28.138948,2018-04-17 21:51:56.778000-04:00
7,7,98.782661,99.921013,99.985039,35.600296,0.083417,-99.755135,2018-04-21 22:47:42.471000-04:00
8,4,98.785606,99.921013,98.174492,35.525597,0.080033,-48.976818,2018-04-23 16:33:57.483000-04:00
9,4,98.785233,99.851280,72.182098,70.275009,0.079571,-73.830124,2018-04-12 12:12:44.545000-04:00


In [141]:
#mood reporter DF

responsesDF = pd.read_csv("../reporter/responses.tsv", sep='\t', header=0)

timeValues = responsesDF.time.values

for x in range(0, len(timeValues)):
    timeValues[x] = dateutil.parser.parse(timeValues[x] + " EDT", tzinfos=tzinfos)

# responsesData.time = timeValues
#responsesData.time = responsesData.time.apply(roundTime)
#responsesData.time.apply(roundTime)

activity_names = list(set(list(map(lambda x: x.lower().strip(), reduce(lambda x,y: x+y, [x.split(",") for x in list(responsesDF.activity.values)])))))

for activity_name in activity_names:
    responsesDF[activity_name.replace(" ", "_") + "_activity"] = responsesDF.activity.apply(lambda x: activity_name in x.lower())

location_names = list(set(list(map(lambda x: x.lower().strip(), reduce(lambda x,y: x+y, [x.split(",") for x in list(responsesDF.location.values)])))))

def split_locations(locations):
    return list(map(lambda x: x.lower().strip(), locations.split(",")))

for location_name in location_names:
    responsesDF[location_name.replace(" ", "_") + "_location"] = responsesDF.location.apply(lambda x: location_name in split_locations(x))
    
del responsesDF['moodNotes']
del responsesDF['trigger']
del responsesDF['activity']
del responsesDF['location']

responsesDF

,time,mood,social,alone,morale,stress,fatigue,compulsions,in_class_activity,errands_activity,...,workshop_activity,leisure_activity,commuting_activity,chatting_activity,just_woke_up_activity,therapy_location,school_location,kevin_home_location,library_location,home_location
0,2018-04-09 10:03:18.315716-04:00,4.0,1,True,4.0,3.0,1.0,True,False,False,...,False,False,False,False,False,False,False,True,False,False
1,2018-04-09 11:03:48.939009-04:00,4.0,1,True,4.0,3.0,1.0,True,False,True,...,False,False,False,True,False,False,False,True,False,False
2,2018-04-09 12:00:17.159027-04:00,3.0,1,True,3.0,2.0,1.0,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,2018-04-09 14:34:53.010916-04:00,3.0,2,True,3.0,2.0,4.0,False,False,True,...,False,True,False,True,False,False,False,True,False,False
4,2018-04-09 16:56:31.979721-04:00,3.0,0,True,3.0,2.0,3.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,2018-04-09 18:00:51.257329-04:00,3.0,0,True,3.0,2.0,4.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
6,2018-04-09 19:58:39.830408-04:00,4.0,2,False,3.0,2.0,4.0,True,True,False,...,False,False,False,True,False,False,True,False,False,False
7,2018-04-10 02:00:04.219220-04:00,4.0,2,False,4.0,2.0,2.0,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,2018-04-10 12:47:00.108283-04:00,3.0,2,False,3.0,3.0,3.0,True,True,False,...,False,False,False,False,False,False,True,False,False,False
9,2018-04-10 13:54:55.984234-04:00,2.0,1,False,2.0,3.0,4.0,True,True,False,...,False,False,False,False,False,False,True,False,False,False


In [103]:
#productivity DF

with open('../rescuetime/productivity.json', 'r') as f:
    productivityFile = json.load(f)

productivityData = productivityFile['rows']

final_productivityData = [];

for x in range(0, len(productivityData)):
    if(productivityData[x][0] > '2018-04-01T90:00:00'):
        time = dateutil.parser.parse(productivityData[x][0] + " EDT", tzinfos=tzinfos)
        prod_score = productivityData[x][4]
        final_productivityData.append((time, prod_score))

productivityDF = DataFrame(final_productivityData, columns=['time', 'productivity_score'])
#productivityDF.time = productivityDF.time.apply(roundTime)
#productivityDF.time.apply(roundTime)

productivityDF

,time,productivity_score
0,2018-04-02 00:00:00-04:00,37.59
1,2018-04-02 01:00:00-04:00,41.02
2,2018-04-02 02:00:00-04:00,52.69
3,2018-04-02 03:00:00-04:00,18.75
4,2018-04-02 04:00:00-04:00,99.68
5,2018-04-02 05:00:00-04:00,93.81
6,2018-04-02 06:00:00-04:00,0.00
7,2018-04-02 07:00:00-04:00,4.16
8,2018-04-02 09:00:00-04:00,0.00
9,2018-04-02 11:00:00-04:00,39.22


In [180]:
# merge all the data

mergedData = []

all_columns = list(responsesDF.columns) + list(keyloggerDF.columns.drop('time')) + list(affectivaDF.columns.drop('time')) + list(productivityDF.columns.drop('time'))

def process_row(row):
    current_time = row['time']

    output_values = list(row.values)

    for other_df in [keyloggerDF, affectivaDF, productivityDF]:
        candidates = other_df[(current_time > other_df['time']) &
                              (current_time - other_df['time'] < datetime.timedelta(hours=3))]
        if candidates.empty:
            return None
        else:
            index_of_max = candidates['time'].argmax()
            candidate = candidates.ix[index_of_max].drop('time')
            output_values += list(candidate.values)
    return output_values

for index, row in responsesDF.iterrows():
    processed_row = process_row(row)
    if processed_row is not None:
        mergedData.append(processed_row)

mergedDF = DataFrame(mergedData, columns = all_columns)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [181]:
mergedDF

,time,mood,social,alone,morale,stress,fatigue,compulsions,in_class_activity,errands_activity,...,Anger_score,Confident_score,blinks,max_attention,max_engagement,max_valence,min_attention,min_engagement,min_valence,productivity_score
0,2018-04-12 00:37:41.151981-04:00,3.0,1,False,3.0,2.0,3.0,False,False,False,...,0.000000,0.000000,2,98.777206,3.767933,0.000000,89.835922,0.081450,-5.812013,50.14
1,2018-04-12 13:40:07.405005-04:00,2.0,0,True,2.0,4.0,2.0,False,False,False,...,0.000000,0.000000,8,98.784813,99.480911,2.589925,96.640442,0.080921,-28.605883,88.91
2,2018-04-12 14:42:51.922735-04:00,2.0,0,True,3.0,4.0,2.0,False,False,False,...,0.000000,0.000000,4,98.766754,1.289316,0.000000,97.681625,0.079623,0.000000,96.85
3,2018-04-12 16:47:51.900184-04:00,2.0,0,True,2.0,4.0,3.0,False,False,False,...,0.000000,0.000000,1,98.785469,99.793770,0.000000,79.901894,0.080212,-66.790077,74.06
4,2018-04-12 17:57:38.743968-04:00,3.0,1,False,3.0,2.0,3.0,False,True,False,...,0.000000,0.000000,1,98.786980,16.004782,0.000000,97.623177,0.079446,0.000000,74.71
5,2018-04-12 21:00:10.756322-04:00,3.0,1,False,3.0,2.0,4.0,False,True,False,...,0.000000,0.000000,0,98.782661,99.917702,0.000000,72.947754,0.079563,-54.905159,30.96
6,2018-04-13 15:27:43.578694-04:00,4.0,0,True,4.0,2.0,3.0,False,False,True,...,0.000000,0.000000,22,98.786911,1.413296,0.000000,97.007210,0.079310,0.000000,54.52
7,2018-04-13 16:29:00.230369-04:00,3.0,1,True,3.0,2.0,3.0,True,False,False,...,0.000000,0.000000,1,98.763466,99.920769,62.067524,37.569420,0.086076,-59.073914,84.55
8,2018-04-13 18:02:39.450067-04:00,3.0,1,True,3.0,2.0,4.0,True,False,False,...,0.000000,0.818552,1,98.783875,99.781586,0.000000,40.514740,0.000000,-23.904726,52.68
9,2018-04-14 17:32:10.595708-04:00,4.0,1,False,4.0,2.0,2.0,True,False,False,...,0.000000,0.000000,0,98.781746,99.920860,99.891769,39.824993,58.210773,0.000000,96.99


In [192]:
mergedDF.corr()

,mood,social,alone,morale,stress,fatigue,compulsions,in_class_activity,errands_activity,eating_activity,...,Anger_score,Confident_score,blinks,max_attention,max_engagement,max_valence,min_attention,min_engagement,min_valence,productivity_score
mood,1.000000,0.295696,-0.079415,0.744266,-0.374536,-0.294927,0.127628,-0.014639,0.191150,0.145050,...,-0.040859,-0.021511,-0.014965,-0.052678,0.007734,0.149499,-0.043312,0.150307,-0.111844,-0.021436
social,0.295696,1.000000,-0.369985,0.088518,-0.297266,0.242956,0.059314,0.364225,-0.076956,-0.041916,...,0.061464,0.194470,-0.101847,-0.066308,0.216434,0.234627,-0.090287,0.020047,-0.212975,-0.111471
alone,-0.079415,-0.369985,1.000000,-0.019784,0.154654,-0.184887,-0.153966,-0.380153,0.047691,-0.164890,...,-0.067786,0.053470,0.212361,0.088515,-0.194006,-0.227092,0.213173,-0.135159,0.123843,-0.061675
morale,0.744266,0.088518,-0.019784,1.000000,-0.360523,-0.371087,0.091682,-0.069975,0.117356,0.078204,...,0.014651,0.088291,-0.089067,-0.086988,-0.080269,0.089671,0.057904,0.153452,0.025971,0.076314
stress,-0.374536,-0.297266,0.154654,-0.360523,1.000000,-0.367367,-0.176462,-0.343872,0.012550,0.128149,...,0.221593,-0.167298,0.078189,0.040308,-0.026129,-0.040685,0.113686,-0.060761,-0.003779,0.263112
fatigue,-0.294927,0.242956,-0.184887,-0.371087,-0.367367,1.000000,0.085397,0.262576,-0.022686,-0.237574,...,-0.226212,0.069275,-0.135667,-0.113515,0.068039,-0.051913,-0.169048,-0.017898,-0.118346,-0.241821
compulsions,0.127628,0.059314,-0.153966,0.091682,-0.176462,0.085397,1.000000,0.014830,0.025258,0.052636,...,-0.095845,0.054257,-0.266932,0.060659,0.139217,-0.071616,-0.248575,0.020266,0.040520,0.171092
in_class_activity,-0.014639,0.364225,-0.380153,-0.069975,-0.343872,0.262576,0.014830,1.000000,-0.082409,-0.067084,...,-0.117564,0.128813,-0.117230,-0.206338,-0.024353,-0.059957,-0.096930,0.141458,-0.054764,-0.125077
errands_activity,0.191150,-0.076956,0.047691,0.117356,0.012550,-0.022686,0.025258,-0.082409,1.000000,0.212508,...,-0.037542,-0.057597,0.266034,0.024330,-0.139939,0.073124,-0.004471,-0.029276,0.006453,-0.015105
eating_activity,0.145050,-0.041916,-0.164890,0.078204,0.128149,-0.237574,0.052636,-0.067084,0.212508,1.000000,...,0.112651,-0.120029,0.120207,0.051662,0.062814,0.060267,0.036512,0.024215,-0.274630,0.056492


In [ ]:
# everything below is garbage

In [162]:
responsesDF

,time,mood,social,alone,morale,stress,fatigue,compulsions,in_class_activity,errands_activity,...,workshop_activity,leisure_activity,commuting_activity,chatting_activity,just_woke_up_activity,therapy_location,school_location,kevin_home_location,library_location,home_location
0,2018-04-09 10:03:18.315716-04:00,4.0,1,True,4.0,3.0,1.0,True,False,False,...,False,False,False,False,False,False,False,True,False,False
1,2018-04-09 11:03:48.939009-04:00,4.0,1,True,4.0,3.0,1.0,True,False,True,...,False,False,False,True,False,False,False,True,False,False
2,2018-04-09 12:00:17.159027-04:00,3.0,1,True,3.0,2.0,1.0,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,2018-04-09 14:34:53.010916-04:00,3.0,2,True,3.0,2.0,4.0,False,False,True,...,False,True,False,True,False,False,False,True,False,False
4,2018-04-09 16:56:31.979721-04:00,3.0,0,True,3.0,2.0,3.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
5,2018-04-09 18:00:51.257329-04:00,3.0,0,True,3.0,2.0,4.0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
6,2018-04-09 19:58:39.830408-04:00,4.0,2,False,3.0,2.0,4.0,True,True,False,...,False,False,False,True,False,False,True,False,False,False
7,2018-04-10 02:00:04.219220-04:00,4.0,2,False,4.0,2.0,2.0,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,2018-04-10 12:47:00.108283-04:00,3.0,2,False,3.0,3.0,3.0,True,True,False,...,False,False,False,False,False,False,True,False,False,False
9,2018-04-10 13:54:55.984234-04:00,2.0,1,False,2.0,3.0,4.0,True,True,False,...,False,False,False,False,False,False,True,False,False,False


In [102]:
productivityDF

,time,productivity_score
0,2018-04-10 03:00:00-04:00,62.74
1,2018-04-10 04:00:00-04:00,90.80
2,2018-04-10 08:00:00-04:00,39.46
3,2018-04-10 09:00:00-04:00,44.46
4,2018-04-10 11:00:00-04:00,76.14
5,2018-04-10 12:00:00-04:00,68.42
6,2018-04-10 13:00:00-04:00,76.75
7,2018-04-10 14:00:00-04:00,83.93
8,2018-04-10 15:00:00-04:00,32.08
9,2018-04-10 16:00:00-04:00,14.42


In [29]:




#merge 

dfs = [keyloggerDF, affectivaDF, responsesData, productivityDF]
df_final = reduce(lambda left, right: pd.merge(left,right,on='time', how='inner'), dfs)

df_final

,tone_name,score,time,blinks,emoji,emotions,max_attention,max_engagement,max_valence,min_attention,...,trigger,activity,social,alone,location,morale,stress,fatigue,compulsions,productivity_score
0,Sadness,0.531304,2018-04-12 13:00:00-04:00,8,😐,,98.784813,99.480911,2.589925,96.640442,...,still stressed about midterm,Schoolwork,0,True,Home,2.0,4.0,2.0,False,88.91
1,Tentative,0.619782,2018-04-12 13:00:00-04:00,8,😐,,98.784813,99.480911,2.589925,96.640442,...,still stressed about midterm,Schoolwork,0,True,Home,2.0,4.0,2.0,False,88.91
2,Analytical,0.620755,2018-04-12 13:00:00-04:00,8,😐,,98.784813,99.480911,2.589925,96.640442,...,still stressed about midterm,Schoolwork,0,True,Home,2.0,4.0,2.0,False,88.91
3,Joy,0.603717,2018-04-12 14:00:00-04:00,4,😐,,98.766754,1.289316,0.000000,97.681625,...,still stressed about midterm,"Schoolwork, Eating",0,True,Home,3.0,4.0,2.0,False,96.85
4,Tentative,0.943283,2018-04-12 17:00:00-04:00,1,😐,,98.786980,16.004782,0.000000,97.623177,...,midterm is in pretty good shape,"Schoolwork, in class",1,False,School,3.0,2.0,3.0,False,74.71
5,Sadness,0.556735,2018-04-13 15:00:00-04:00,1,"😐,😛,😳,😗,☺,😜,😡","contempt,disgust,joy",98.763466,99.920769,62.067524,37.569420,...,so much exercise,"Errands, Resting, Leisure, Schoolwork",0,True,Home,4.0,2.0,3.0,False,54.52
6,Joy,0.678779,2018-04-13 15:00:00-04:00,1,"😐,😛,😳,😗,☺,😜,😡","contempt,disgust,joy",98.763466,99.920769,62.067524,37.569420,...,so much exercise,"Errands, Resting, Leisure, Schoolwork",0,True,Home,4.0,2.0,3.0,False,54.52
7,Tentative,0.701639,2018-04-13 15:00:00-04:00,1,"😐,😛,😳,😗,☺,😜,😡","contempt,disgust,joy",98.763466,99.920769,62.067524,37.569420,...,so much exercise,"Errands, Resting, Leisure, Schoolwork",0,True,Home,4.0,2.0,3.0,False,54.52
8,Joy,0.623108,2018-04-13 16:00:00-04:00,1,"😗,😐,😉",,98.783875,99.781586,0.000000,40.514740,...,can't debug code,Schoolwork,1,True,Home,3.0,2.0,3.0,True,84.55
9,Joy,0.623108,2018-04-13 16:00:00-04:00,3,"😐,😳",,98.785866,54.516327,15.326530,63.716442,...,can't debug code,Schoolwork,1,True,Home,3.0,2.0,3.0,True,84.55


In [26]:
keyloggerDF[(keyloggerDF['time'] > datetime.datetime(2018,4,28)) & (keyloggerDF['time'] < datetime.datetime(2018,4,30))]

,tone_name,score,time
362,Joy,0.598466,2018-04-28 21:00:00-04:00
363,Sadness,0.600363,2018-04-28 21:00:00-04:00
364,Tentative,0.885297,2018-04-28 21:00:00-04:00
365,Analytical,0.577231,2018-04-28 21:00:00-04:00
366,Tentative,0.647781,2018-04-29 00:00:00-04:00
367,Tentative,0.948691,2018-04-29 02:00:00-04:00
368,Tentative,0.808202,2018-04-29 05:00:00-04:00
369,Joy,0.529440,2018-04-29 16:00:00-04:00
370,Sadness,0.544247,2018-04-29 16:00:00-04:00
371,Tentative,0.814426,2018-04-29 16:00:00-04:00


In [27]:
affectivaDF[(affectivaDF['time'] > datetime.datetime(2018,4,28)) & (affectivaDF['time'] < datetime.datetime(2018,4,30))]

,blinks,emoji,emotions,max_attention,max_engagement,max_valence,min_attention,min_engagement,min_valence,time
14,0,"😐,😳,😗,☺,😞",joy,98.781281,99.915054,99.175507,37.960567,0.000000,-99.991234,2018-04-28 22:00:00-04:00
19,1,"😱,😐,😛,😞,😏",contempt,98.785873,99.894897,3.636924,76.914368,0.080012,-78.877396,2018-04-28 23:00:00-04:00
23,0,"😗,😐",,98.779549,49.165134,0.000000,84.595108,0.079385,0.000000,2018-04-29 01:00:00-04:00
27,0,"😱,😐,😛,😳","disgust,surprise",98.786453,99.915482,16.350876,71.258888,0.079548,-57.315376,2018-04-29 02:00:00-04:00
31,0,😐,,98.773895,2.576165,0.000000,96.804039,0.079389,-12.074724,2018-04-29 14:00:00-04:00
34,5,"😗,☺,😐,😛",joy,98.349716,99.855766,60.574188,39.735512,0.080418,-84.866615,2018-04-29 15:00:00-04:00
37,2,😐,,98.786430,91.064117,0.000000,94.988678,0.079215,-88.536125,2018-04-29 16:00:00-04:00
40,7,"😱,☺,😐,😗","disgust,joy,surprise",98.742371,99.920341,97.010284,37.979870,0.079232,-75.663185,2018-04-29 18:00:00-04:00
127,0,😐,,98.702408,2.466208,0.000000,95.001732,0.079414,0.000000,2018-04-28 20:00:00-04:00
